In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectsSimulatorCollection
from shared.analyzers import FixedEffectsAnalyzer, AnalyzerCollection
import matplotlib.pyplot as plt

In [2]:
N = 50
T = 100
alpha = 0.05

effect_simulators = EffectsSimulatorCollection()
effect_simulators.add_effect_simulator(ConstantEffectSimulator("No Effect", N, 0., 0., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Shared Random Effect", N, 0., 1., 0.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Distinct Random Effects", N, 0., 1., 1.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("Distinct Random Effects", N, 0., 1., 1., epsilon_sd=1.))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Moderated Effects", N, 0., 1., 1., 0.5, 0.2, epsilon_sd=1.))
effect_simulators.add_effect_simulator(ConstantEffectSimulator("No Effect", N, 0., 0., 0.))

analyzers = AnalyzerCollection()
analyzers.add_analyzer(FixedEffectsAnalyzer("Fixed Effects", alpha))

for effect_simulator in effect_simulators:
    effect_simulator.sample_params()
    data_simulator = DataSimulator(N, T).simulate(effect_simulator)
    for analyzer in analyzers:
        results = analyzer.analyze(data_simulator)

In [3]:
results.get_mean_series()

mean_estimate    0.001703
mean_lower       0.001695
mean_upper       0.001711
dtype: float64